# Import packages

In [1]:
from functools import partial

import numpy as np
import matplotlib.pyplot as mplt

from pycolleff.longitudinal_equilibrium import ImpedanceSource, \
    LongitudinalEquilibrium
import pycolleff.rings.sirius as sirius
import pycolleff.impedances as imp
import pycolleff.materials_params as mat_par

# Create a resistive wall impedance to be used as example

In [2]:
cu_cond = mat_par.cu_cond
cu_rel_time = mat_par.cu_rel_time
neg_cond = mat_par.neg_cond
neg_rel_time = mat_par.neg_rel_time
neg_thick = mat_par.neg_thick
ndfe_cond = mat_par.ndfe_cond
ndfe_mur = mat_par.ndfe_mur

energy = 3e9  # [eV]
length = 500  # [m]

epb = np.array([1, 1, 1, 1, 1])
mub = np.array([1, 1, 1, 1, ndfe_mur])
ange = np.array([0, 0, 0, 0, 0])
angm = np.array([0, 0, 0, 0, 0])
sigmadc = np.array([0, neg_cond, cu_cond, 0, ndfe_cond])
tau = np.array([0, neg_rel_time, cu_rel_time, 0, 0])

radius = 12e-3 + np.array([-neg_thick, 0, 1e-3, 3e-3])

ang_freq = imp.get_default_reswall_w(radius=radius[0], energy=energy)

epr, mur = imp.prepare_inputs_epr_mur(
    ang_freq, epb, mub, ange, angm, sigmadc, tau
)

Zll, Zdx, Zdy = imp.multilayer_round_chamber(
    ang_freq,
    length,
    energy,
    epr,
    mur,
    radius,
    precision=70,
    wmax_arb_prec=1e12,
    arb_prec_incl_long=False,
    print_progress=True,
)

Zll, ang_freq = imp.get_impedance_for_negative_w(
    Zll, ang_freq, impedance_type='ll'
)
Zdx = imp.get_impedance_for_negative_w(Zdx, impedance_type='t')
Zdy = imp.get_impedance_for_negative_w(Zdy, impedance_type='t')

generic = ImpedanceSource()
generic.zl_table = Zll
generic.ang_freq_table = ang_freq

# Methods: Impedance or Wake
generic.calc_method = ImpedanceSource.Methods.ImpedanceDFT

# Device type: Active of Passive
generic.active_passive = ImpedanceSource.ActivePassive.Passive

0000/0302 -> freq =        0.1  (ET: 0.11 s)
0001/0302 -> freq =       0.11  (ET: 0.03 s)
0002/0302 -> freq =       0.12  (ET: 0.03 s)
0003/0302 -> freq =       0.13  (ET: 0.03 s)
0004/0302 -> freq =       0.15  (ET: 0.03 s)
0005/0302 -> freq =       0.16  (ET: 0.03 s)
0006/0302 -> freq =       0.17  (ET: 0.03 s)
0007/0302 -> freq =       0.19  (ET: 0.03 s)
0008/0302 -> freq =       0.21  (ET: 0.03 s)
0009/0302 -> freq =       0.23  (ET: 0.05 s)
0010/0302 -> freq =       0.25  (ET: 0.05 s)
0011/0302 -> freq =       0.28  (ET: 0.03 s)
0012/0302 -> freq =       0.31  (ET: 0.03 s)
0013/0302 -> freq =       0.34  (ET: 0.03 s)
0014/0302 -> freq =       0.37  (ET: 0.03 s)
0015/0302 -> freq =        0.4  (ET: 0.03 s)
0016/0302 -> freq =       0.44  (ET: 0.03 s)
0017/0302 -> freq =       0.49  (ET: 0.03 s)
0018/0302 -> freq =       0.53  (ET: 0.04 s)
0019/0302 -> freq =       0.59  (ET: 0.06 s)
0020/0302 -> freq =       0.64  (ET: 0.05 s)
0021/0302 -> freq =       0.71  (ET: 0.06 s)
0022/0302 

# Define methods

In [7]:
def calculate_longeq_single_bunch_vs_current(
    ring, impedance_sources, currents
):
    """Calculate longitudinal equilibrium."""
    print('Calculating Longitudinal Equilibrium...')
    fillp = np.zeros(ring.harm_num, dtype=float)
    fillp[0] = 1
    longeq = LongitudinalEquilibrium(
        ring=ring,
        impedance_sources=impedance_sources,
        fillpattern=fillp * 1e-4,
    )
    longeq.feedback_on = False
    longeq.zgrid = np.linspace(-1, 1, 2001) * ring.rf_lamb / 2
    longeq.max_mode = 1000*ring.harm_num
    longeq.min_mode0_ratio = 1e-10  # criteria for convergence

    bunlen, syncfreq = [], []
    for current in currents:
        longeq.fillpattern = fillp * current
        _ = longeq.calc_longitudinal_equilibrium(
            niter=1000, tol=1e-8, beta=0.1, m=3, print_flag=True)
        hcav = longeq.impedance_sources[0]
        hvolt = longeq.calc_induced_voltage_wake(
            dist=longeq.distributions, wake_source=hcav)
        mvolt = longeq.main_voltage
        tvolt = mvolt + hvolt
        syncfreq.append(longeq.calc_synchrotron_frequency(
            tvolt, method="action", max_amp=5, nrpts=100
        )['avg_sync_freq'])
        dist, _ = longeq.calc_distributions_from_voltage(voltage=tvolt)
        _, sigmaz = longeq.calc_moments(longeq.zgrid, dist[0])
        bunlen.append(sigmaz[0])
    return bunlen, syncfreq


def calc_mode_coupling(
    ring,
    ang_freq,
    impedance,
    currents,
    plane='x',
    max_azi=4,
    max_rad=20,
    cbmode=0,
    use_fokker=True,
    bunlen_vs_current=None,
):
    """."""
    bunlen0 = ring.bunlen
    curr0 = ring.total_current

    dtunes = []
    mod_mat = fok_mat = None
    for i, cur in enumerate(currents):
        ring.total_current = cur
        if bunlen_vs_current is not None:
            ring.bunlen = np.interp(
                cur, bunlen_vs_current[0], bunlen_vs_current[1])
            mod_mat = None
        dtune, mod_mat, fok_mat = ring.transverse_mode_coupling(
            w=ang_freq,
            Zt=impedance,
            plane=plane,
            max_azi=max_azi,
            max_rad=max_rad,
            cbmode=cbmode,
            use_fokker=use_fokker,
            modecoup_matrix=mod_mat,
            fokker_matrix=fok_mat,
        )
        dtunes.append(dtune)
        print(
            f'{i+1:02d}/{len(currents):02d}: ' +
            f'Ib = {cur/ring.num_bun*1e3:7.3f} mA,   Grow Rate = ' +
            f'{dtune.imag.max()*ring.sync_tune*ring.rev_ang_freq:7.0f} 1/s'
        )

    ring.bunlen = bunlen0
    ring.total_current = curr0

    dtunes = np.array(dtunes)
    dtunes *= ring.sync_tune
    gr = dtunes.imag * ring.rev_ang_freq
    gr = np.sort(gr)
    tushif = np.sort(dtunes.real, axis=1)

    return gr, tushif


SyntaxError: positional argument follows keyword argument (3340033025.py, line 10)

# Define the ring model

In [4]:
ring = sirius.create_ring()
ring.en_lost_rad = 870e3
ring.gap_voltage = 3.0e6
ring.total_current = 350e-3
ring.chromx = 2.5
ring.chromy = 2.5
print(ring)

Lattice Version             :   SI.v25.01-s05.02  
Circumference [m]           :       518.387       
Revolution Period [us]      :        1.729        
Revolution Frequency [kHz]  :       578.318       
Energy [GeV]                :        3.000        
U0 [keV]                    :       870.000       
Vgap [MV]                   :        3.000        
Momentum Compaction         :       1.63e-04      
Harmonic Number             :         864         
Current [mA]                :       350.000       
Current per Bunch [mA]      :        0.405        
Synchrotron Tune            :       0.00356       
Tunes x/y                   :    49.078/14.137    
Chromaticities x/y          :     2.500/2.500     
Damping Times x/y/e [ms]    :   16.9/ 22.0 /12.9  
Energy Spread [%]           :        0.0887       
Bunch Length [mm]           :        3.250        



# Calculate longitudinal equlibrium

This step is important to model de dependency of bunch length and synchrotron frequency as function of the current.

In [6]:
currents = np.linspace(0, 20) * 1e-3
bunlen, syncfreq = calculate_longeq_single_bunch_vs_current(
    ring, [generic, ], currents
)

Calculating Longitudinal Equilibrium...


AttributeError: 'NoneType' object has no attribute 'size'

# Calculate transverse mode-coupling

In [ ]:
grow, freqshit = calc_mode_coupling(
    ring,
    ang_freq=ang_freq,
    impedance=Zdx,
    currents=currents,
    plane='x',
    max_azi=4,
    max_rad=20,
    use_fokker=True,
    bunlen_vs_current=(currents, bunlen),
)